# The method to load circuits in `QASM` format.


In [ ]:
import csv
from timeit import default_timer as timer
from typing import Callable, List, Tuple

import numpy as np
from mitiq import zne
from qiskit import transpile
from skquant.opt import minimize

from global_settings import Context
from vqe_utils import get_vqe_circuit


def _get_group_exp(ctx: Context, count_general, pauli):
    pauli = pauli[::-1]
    expectation_val = 0
    num = pauli.count("I")
    num = ctx.num_qubits - num
    I_pos = []
    M_pos = []
    # 记录测量的位置
    for ind in range(len(pauli)):
        if pauli[ind] != "I":
            M_pos.append(ind)
        else:
            I_pos.append(ind)
    count = {}
    for i in range(2**num):
        ind = 0
        p_str = list("000000000000")
        bi_i = bin(i)[2:]
        bi_i = bi_i.rjust(num, "0")
        bi_i = list(bi_i)
        for j in M_pos:
            p_str[j] = bi_i[ind]
            ind += 1
        p_str = "".join(p_str)
        count[p_str] = 0
    for pau in count_general.keys():
        p_str_g = list(pau)
        for i in I_pos:
            p_str_g[i] = "0"
        p_str_g = "".join(p_str_g)
        count[p_str_g] += count_general[pau]
    expectation_val, _ = ctx.readout_mitigator.expectation_value(count)
    return expectation_val


def _count_to_exps(ctx, count, grouped_paulis):
    ans = []
    for pauli in grouped_paulis:
        if pauli == len(pauli) * "I":
            ans.append(1.0)
        else:
            mitigated = _get_group_exp(ctx, count, pauli)
            ans.append(mitigated)
    return ans


def get_pauli_expectations(
    ctx: Context,
    grouped_paulis,
    fold: Callable,
) -> List[float]:
    # TODO: Pauli grouping with Qiskit.
    """Compute the expectations of each Pauli string.

    Parameters
    ----------
    ctx : Context
        Context with configuration and meta information of the application.
    paulis : List[str]
        Pauli strings constituting the Hamiltonian.
    params : np.ndarray
        Parameters in quantum circuit of VQE.

    Returns
    -------
    List
        Returns a list, where each element represents the expected value
        of the measurement with the corresponding Pauli string.
    """
    '''circuit = get_vqe_circuit(
        ctx.num_qubits, "ZZZZZZZZZZZZ", **ctx.vqe_kwargs
    )
    circuit = transpile(
        circuit,
        ctx.system_model,
        optimization_level=3,
        seed_transpiler=ctx.seed,
    )'''
    #ac, 这里需要load进circuit
    circuit=ctx.circuit #add, ac, 109
    scaled_circuits = [fold(circuit, scale) for scale in ctx.zne_scale]
    counts = [
        ctx.noisy_simulator.run(circuit, shots=ctx.shots).result().get_counts()
        for circuit in scaled_circuits
    ]
    scaled_expectations = []

    for count in counts:
        exp = _count_to_exps(ctx, count, grouped_paulis)
        scaled_expectations.append(exp)
    zne_exps = [
        zne.RichardsonFactory.extrapolate(
            ctx.zne_scale, [exp[i] for exp in scaled_expectations]
        )
        for i in range(len(scaled_expectations[0]))
    ]

    return zne_exps


def run_vqe_iter(
    ctx: Context,
) -> float:
    """Compute the ground state energy in each iteration.

    Parameters
    ----------
    ctx : Context
        Context with configuration and meta information of the application.
    params : np.ndarray
        Parameters in quantum circuit of VQE.

    Returns
    -------
    float
        The computed energy in each iteration.
    """
    start = timer()
    # NOTE: We only compute the important part.
    expectations = get_pauli_expectations(
        ctx,
        grouped_paulis=ctx.hamiltonian.paulis,
        fold=ctx.zne_fold,
    )
    end = timer()
    energy = np.inner(ctx.hamiltonian.coefs, expectations)
    print(f"Energy computed by VQE is {energy}, in {end - start}s.")

    '''with open(f"{ctx.save_dir}/energy_log.csv", "a") as file:
        writer = csv.writer(file)
        writer.writerow([energy])'''

    return energy


def run_vqe(
    ctx: Context,
) -> float:
    """Run VQE instance to calculate the ground
    state energy of the hydroxyl cation.

    Parameters
    ----------
    ctx : Context
        Context with configuration and meta information of the application.

    Returns
    -------
    Tuple[float, np.ndarray]
        Return the caculated ground state energy of ·OH and
        the best parameters during VQE iterations.
    """
    #params_guess = np.array(ctx.prepared_cafqa_params) * np.pi / 2.0
    #no parameters? ac, 109, Left
    energy_vqe = run_vqe_iter(ctx),
    return energy_vqe


Ctx = NamedTuple(
    "Context",
    [
        (
            "readout_mitigator",
            Union[LocalReadoutMitigator, CorrelatedReadoutMitigator],
        ),
        ("num_qubits", int),
        ("noisy_simulator", AerSimulator),
        ("system_model", Union[FakeCairo, FakeKolkata, FakeMontreal]),
        ("save_dir", Path),
        ("seed", int),
        ("shots", int),
        ("hamiltonian", Hamiltonian),
        ("vqe_kwargs", dict[str, Any]),
        ("zne_scale", list[float]),
        ("zne_fold", Callable[[QuantumCircuit, list[float]]]),
        ("circuit",QuantumCircuit)
    ],
)

def get_ctx(noise_model,seed,circuit):
    args={"noise":noise_model,"threshold":1.0,"seed":seed,"save":"./result","readout_mitigator":"correlated","shots":6000,"zne_scale":[1.0,2.0,3.0],"zne_fold":"global" }
    
    if noise_model=="montreal":
        pass 
    elif noise_model=="cairo":
        pass
    elif noise_model=="kolkata":
        pass
    else:
        raise("Error! The noise model passed in shall be one of: cairo, kolkata, montreal!")

    noisy_simulator = _load_noise(args["noise"])
    (
        coefs,
        paulis,
        HF_bitstring,
        num_qubits,
    ) = _get_quantum_info()
    coefs, paulis = _simplify_paulis(
        _find_important_terms(coefs, args["threshold"]), coefs, paulis
    )
    _seed_everything(args["seed"])
    save_dir = Path(args["save"])
    save_dir.mkdir(exist_ok=True)
    vqe_kwargs = {
        "ansatz_reps": 2,
        "init_last": False,
        "HF_bitstring": HF_bitstring,
    }
    system_model = FakeMontreal()
    return Ctx(
        readout_mitigator=_get_readout_mitigator(
            args["readout_mitigator"], num_qubits, noisy_simulator
        ),
        num_qubits=num_qubits,
        noisy_simulator=noisy_simulator,
        system_model=system_model,
        save_dir=save_dir,
        seed=args["seed"],
        shots=args["shots"],
        hamiltonian=Hamiltonian(coefs=coefs, paulis=paulis),
        vqe_kwargs=vqe_kwargs,
        zne_scale=args["zne_scale"],
        zne_fold=fold_global
        if args["zne_fold"] == "global"
        else fold_gates_at_random, #ac, doubt
        circuit=circuit, #store the QuantumCircuit passed in
    )

    def test(noise_model,seed):
        dir="QASM_circuits"+noise_model+".qasm"
        circuit=qiskit.qasm2.load(dir)
        ctx=get_ctx(noise_model,seed,circuit)
        energy = run_vqe(ctx)




# Explanation to the Optimization Techniques


## CAFQA method



## Pauli String Grouping Technique
### Abstract
One of the main methods proposed to suppress an increase in the number of measurements is the partitioning method, where Pauli strings are partitioned so that their expectation values can be measured simultaneously. 

In this method, a group of Pauli strings is divided into subgroups, and all the components of each subgroup are measured simultaneously using only one circuit (hereafter, such subgroups are called “partitions”). 

### Implementation
The problem of Pauli string grouping can be broken down into two parts:
1. The partition of a given group of Pauli strings.
2. The method to get the equivalent measurement results of pauli strings in a partition from the result of the so-called "simultaneous measurement" 

We solve the first part (the partition) of the problem by (?xxx L) calling the function/method in the `Estimator`, and we tackle the second part by using the function xxx in xxx.py

this is the workflow of xxx:
1. xxx



## Error Mitigation Technique


### Readout Error Mitigation
#### Abstract
Readout-Error Mitigation (REM) is an error mitigation technique in which inverted transition / confusion matrices are applied to the noisy measurement results.

technique is based on two main ideas:

1. Generating a confusion matrix for a specific device;

2. Computing the psuedoinverse of this confusion matrix and applying it to the raw measurement (or “readout”) results.

#### Implementation
##### Measurement based method
The readout mitigator is generated from an assignment matrix: a $2^n \times 2^n$ matrix A such that $A_{y,x}$ is the probability to observe y given the true outcome should be x. The assignment matrix is used to compute the mitigation matrix used in the readout error mitigation process itself.

A Correlated readout mitigator uses the full $2^n \times 2^n$ assignment matrix, meaning it can only be used for small values of n. 

We use correlated readout mitigation in our program for better accuracy.

we can get the mitigator(derived from assignment matrix) by the following measurement and process procedure:

```python
exp = LocalReadoutError(qubits)
for c in exp.circuits():
    print(c)
result = exp.run(backend)
mitigator = result.analysis_results(0).value
```
After we get the mitigator, it's easy to calculate the mitigated expectation value of a diagonal observable (namely a Pauli string) from the measurement result:
`expectation=mitigator.expectation_value(data=measurement_count, diagonal=pauli,  shots=shots)`


##### Noise model based method
the figure below shows how to use REM by mitiq.rem.
![Alt text](image.png)

Since the estimated confusion matrix A is circuit-independent—it characterizes the readout noise of the device regardless of what circuit is being executed, we can get matrix A directly from the noise model.

We can obtain the mitigator by: `mitigator = qiskit.result.CorrelatedReadoutMitigator(assignment_matrices=matrix_A)`

### Zero-Noise Extrapolation (ZNE)
The figure below shows the main procedure of ZNE mitigation method:
![Alt text](image-1.png)

#### Abstract
Zero noise extrapolation (ZNE) is an error
mitigation technique used to extrapolate the noiseless expectation value of an
observable from a range of expectation values computed at different noise levels.
This process works in two steps:

- **Step 1: Intentionally scale noise**. This can be done with different methods.
*Pulse-stretching*  can be used to increase the noise level of a quantum computation. Similar results can be obtained, at a gate-level, with *unitary folding* or *identity insertion scaling*.

- **Step 2: Extrapolate to the noiseless limit**. This can be done by
fitting a curve (often called *extrapolation model*) to the expectation values measured at different noise levels to extrapolate the noiseless expectation value.

#### Implementation
##### First Step: Generating noise-scaled quantum circuits
  - We provide a `QPROGRAM`, i.e., a `QuantumCircuit` defined by Qiskit.
  - Mitiq generates a set of noise-scaled circuits by applying a scaling method (*unitary folding* or *identity insertion scaling*) with different scale factors.
  - The noise-scaled circuits are executed on the noisy backend obtaining a set of noise-scaled expectation values.

##### Second Step: Inferring the zero-noise value from the measured results
  - A parametrized curve is fit to the noise-scaled expectation values obtained in step one.
  - The curve is extrapolated to the zero-noise limit, obtaining an error mitigated expectation value.

##### The optimization of parameters for ZNE
There are three key parameters in ZNE that can affect its performance: 
- the scale factors, i.e., [1.0, 2.0, 3.0] (which is the optimal choice, from our previous experiment)
- the folding method (which inserts gates into original circuit to scale the noise), i.e., the fold_global method (which, from our previous experiment, is also the optimal choice)
- the extrapolation method, i.e., Rechardson extrapolation method (which is the optimal choice, from our previous experiment)



## Transpile Optimization
We flexibly make use of the transpile optimization to improve the performance of our program. 
The function we called to optimize the circuit transpiling is xxx.transpile(optimization_level=xxx)
here we set the otimization_level to 3 in certain parts of our procedure, to mitigate the extent of the circuit decoherence and quantum gate noise.
and in some other parts of the process, we set optimization_level to 0 with the aim of avoiding disturbing the noise scaled quantum circuit, otherwise the inserted gates pair like $X$ and $X^{\dagger}$ which is equivalent to an identity gate in theory but invokes noise in practice.


# Alignment with the qiskit estimator
